In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
sns.set()
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
df = pd.read_csv("kidney_disease.csv")

In [ ]:
df.head(10)

In [ ]:
df['classification'].value_counts()

In [ ]:
df.shape